In [26]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import slogdet
import random
from random import random
from scipy.stats import multivariate_normal
from scipy.stats import mode
import multiprocessing as mP
import time

import concurrent.futures as cF
from functools import partial


### Functions

###### Generates Data

In [46]:
def generate_mat(T, dim_y, p_s, noise):
    
    p_ns = 1 - p_s
    # Get dimension of vectorized matrix
    dim_x = dim_y**2
    
    # Initialize coefficient and adjacency matrices
    C = np.random.rand(dim_y, dim_y)*1-0.5
    A = np.ones([dim_y, dim_y])
    
    # Generate Matrices
    for i in range(dim_y):
        cond = True
        while(cond):
            for j in range(dim_y):
                if(i==j):
                    p = p_s
                else:
                    p = p_ns
                if(random() >= p):
                    A[i,j] = 0
                else:
                    A[i,j] = 1
                if(sum(A[i,:])!=0):
                    cond = False
    # Get matrix topology
    C = C*A
    
    # Generate the time series
    y = np.zeros([dim_y,T])
    y[:,0] = np.random.rand(1,dim_y)
    for t in range(1,T):
        y[:,t] = np.dot(C,y[:,t-1]) + np.random.multivariate_normal([0]*dim_y, noise*np.identity(dim_y), 1)   
        
    return A, C, y, dim_x

###### Computes posterior of matrix normal

In [47]:
def mn_conjugate_var(y, var_y, mu_0, sig_0):
    
    # Obtain dimensions of observations
    dim_y = len(y[:,0])
    
    # Obtain length of time series
    T = len(y[0,:])
    
    # Obtain Matrix Normal parameters of likelihood
    y1 = np.dot(y[:,0:T-1], y[:,0:T-1].T)
    y2 = np.dot(y[:,0:T-1],y[:,1:T].T)
    
    M = inv(y1)*y2.T
    U = var_y*inv(y1)
    V = np.identity(dim_y)
    
    # Convert to vector form
    mu_x = M.flatten()
    sig_x = np.kron(V,U)
    dim_x = dim_y^2
    
    # Compute the posterior of vectorized matrix 
    sig = inv( inv(sig_0) + inv(sig_x) )
    mu1 = np.dot(inv(sig_0),mu_0)
    mu2 = np.dot(inv(sig_x),mu_x)
    mu = np.dot(sig, mu1 + mu2)
    
    return mu, sig, mu_x, sig_x

In [4]:
def logmvnpdf(x, mu, SIGMA):
    D = len(x)
    const = -0.5*D*np.log(2*np.pi)
    xc = x - mu
    term1 = -0.5*sum(np.dot(xc,inv(SIGMA))*xc)
    sign_sig, logdet_sig = slogdet(SIGMA)
    logdet_sig = sign_sig*logdet_sig
    term2 = const - 0.5*logdet_sig
    logp = term1 + term2   
    return logp
    

In [5]:
def mode_adj_matrix(M):
    dim_y = len(M)
    M_mode = np.zeros([dim_y,dim_y])
    for j in range(dim_y):
        for k in range(dim_y):
            n =  mode(M[j,k,:])
            M_mode[j,k] = n[0]
    return M_mode

In [6]:
def gibbs_bernoulli(gamma, parameters):
    
    I, I0, thin_rate, A_init, C, mu, sig, A = parameters
    # Obtain dimensions 
    dim_y = len(C[:,0])
    
    # Initialize arrays to store samples
    A_s = np.zeros([dim_y, dim_y, I+1])
    A_old = np.zeros([dim_y, dim_y])
    
    # Set the initial states of the Markov chains
    A_s[:,:,0] = A_init
    A_old = A_s[:,:,0]
    
    # For loop for Gibbs
    for i in range(1,I):
        for j in range(dim_y):
            for k in range(dim_y):

                # Conside that A[j,k] = 1
                A_old[j,k] = 1
                C_temp = C*A_old
                log_pa1 = np.log(gamma) + logmvnpdf(C_temp.flatten().T, mu.T, sig)

                # Conside that A[j,k] = 0
                A_old[j,k] = 0
                C_temp = C*A_old
                log_pa0 = np.log(1-gamma) + logmvnpdf(C_temp.flatten().T, mu.T, sig)

                # Sample the topology
                pa0 = np.exp(log_pa0 - np.max([log_pa0, log_pa1]))
                pa1 = np.exp(log_pa1 - np.max([log_pa0, log_pa1]))
                prob_1 = pa1/(pa1 + pa0)

                A_old[j,k] = int(random()<prob_1)
        
        # Store sample from iteration i
        A_s[:,:,i] = A_old
    
    # Apply burn-in
    A_s = A_s[:,:,I0+1:I+1]
    A_s = A_s[:,:,::thin_rate]
    
    # Find estimate of adjacency matrix from obtained Gibbs samples
    A_est = mode_adj_matrix(A_s)
    
    fs = f_score(A, A_est)
    
    return A_est, fs
    

In [7]:
def f_score(A, A_hat):
    precision = sum(sum(A_hat*A))/sum(sum((A_hat==1).astype(int)));
    recall = sum(sum(A_hat*A))/sum(sum((A==1).astype(int)));
    fscore = 2*(precision*recall/(precision+recall));
    return fscore
    

# Main Script

Generating Data

In [44]:
# Dimension of data
dim_y = 4

# Sparsity percentage
p_s = 0.7 
p_ns = 1 - p_s

# Observations (Length of time series)
T = 10000

# Observation noise
var_u = 1

true = generate_mat(T, dim_y, p_s, var_u)

A, C, y, dim_x = true

Bayesian Ridge Regression

In [45]:
# Mean
mu_0 = [0]*dim_x

# Covariance
var_0 = 1
sig_0 = var_0*np.identity(dim_x)

# Obtain the posterior of the vectorized matrix
estimates = mn_conjugate_var(y, var_u, mu_0, sig_0)

mu_c, sig_c, mu_x, sig_x = estimates

# Convert estimate to matrix form
C_est = mu_c.reshape(dim_y, dim_y)

# Compute MSE
MSE = sum(sum((C-C_est)**2))/dim_x
MSE

0.02180651747024084

Gibbs Sampler - Bernoulli Prior

In [39]:
# Gibbs Sampler Settings

# Number of iterations
I = 4000

# Burn in period
I0 = 2000

# Thinin parameter
K = 2

# Initial Adjacency Matrix
A_init = np.ones([dim_y,dim_y])

# Hyper prior range
#gamma = list(np.linspace(0.1,0.8, num=8))
gamma_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Number of runs
R = 1

Call Gibbs Sampler

In [40]:
gibbs_settings = (I, I0, K, A_init, C_est, mu_x, sig_x, A)

In [41]:
with cF.ProcessPoolExecutor() as executor:

    sub_gibbs = partial(gibbs_bernoulli, parameters=gibbs_settings)
    results = executor.map(sub_gibbs, gamma_list)
    
    test_res = list(results)
    for est in test_res:
        print(est)
        
    

(array([[0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]]), 0.4)
(array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]]), 0.5454545454545454)
(array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]]), 0.5454545454545454)
(array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]]), 0.5454545454545454)
(array([[1., 0., 1., 1.],
       [0., 1., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 1.]]), 0.7058823529411765)
(array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 0.6666666666666666)
(array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 0.6666666666666666)
(array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]]), 0.6666666666666666)


In [33]:
for prior in [0.1, 0.5]:
    results_reg = gibbs_bernoulli(prior, gibbs_settings)
    print(results_reg[1])

0.4
0.631578947368421


In [42]:
C_est

array([[-5.32652644e-02,  1.48038624e-03,  1.73301774e-03,
         4.24240794e-04],
       [ 2.16781122e-03, -4.68014053e-01,  3.21940587e-04,
         2.50488417e-04],
       [ 1.82940124e-02,  3.98464751e-04,  3.15839697e-01,
        -1.18568133e-03],
       [-2.51630972e-04, -4.52561736e-04,  7.89874360e-03,
         2.21350057e-02]])

In [43]:
C

array([[-0.03072427,  0.01853766,  0.        ,  0.36982851],
       [-0.        , -0.4788122 ,  0.        ,  0.        ],
       [-0.43113228, -0.        ,  0.38195141,  0.        ],
       [-0.18283932, -0.        , -0.13330547, -0.        ]])

In [17]:
results_reg[0]

array([[1., 1.],
       [1., 1.]])

In [18]:
A

array([[1., 0.],
       [0., 1.]])

In [37]:
np.exp(-0.005)

0.9950124791926823

In [38]:
np.exp(np.float64(-0.005))

0.9950124791926823